In [5]:

import json
import pprint
import time
import threading
import re


from datetime import datetime

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
#browser = webdriver.Chrome('C:\\path\\to\\chromedriver.exe')
def Mbox(title, text, style):
    return ctypes.windll.user32.MessageBoxW(0, text, title, style)

import ctypes  # An included library with Python install.

html = None

urls = {
    "https://www.tesla.com/inventory/used/ms?TRIM=MSPLAID,MSAWD,MSPERF,75D&arrangeby=plh&zip=94555",
    "https://www.tesla.com/inventory/used/m3?arrangeby=plh&zip=94555",
    "https://www.tesla.com/inventory/used/mx?TRIM=75D&arrangeby=plh&zip=94555",
    "https://www.tesla.com/inventory/used/my?arrangeby=plh&zip=94555"
}

results_container_selector = 'div.results-container.results-container--grid.results-container--has-results'
delay = 10  # seconds

#priceThreshold = 1000000000000
car={'price':[], 'colour':[],'car_type':[],'year':[],'trim':[],'mileage':[],'location':[],'wheels':[],'interior':[] }


for city in urls:

    try:

        print(datetime.now().strftime("%H:%M:%S") + " Searching Tesla's website in " + city)
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        browser = webdriver.Chrome('C:/Users/STSC/OneDrive - horizon.csueastbay.edu/Documents/Semester II/Data Analytics/Project 2023/chromedriver.exe', options=chrome_options)
        browser.get(city)

        # wait for results to be displayed
        WebDriverWait(browser, delay).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, results_container_selector))
        )

    except TimeoutException:
        print('Loading took too much time!')
    else:
        html = browser.page_source
    finally:
        browser.quit()

    if html:
        soup = BeautifulSoup(html, 'lxml')
        
        for car_html in soup.select_one(results_container_selector).findChildren('article'):

           
            try:
                #car['price'] = int(re.sub('[^0-9]','', car_html.select_one('section.result-header').select_one('div.result-pricing').select_one('div.result-price').select_one('span.result-purchase-price.tds-text--h4').text.replace('$', '').replace(',', '')))
                price = int(re.sub('[^0-9]','', car_html.select_one('section.result-header').select_one('div.result-pricing').select_one('div.result-price').select_one('span.result-purchase-price.tds-text--h4').text.replace('$', '').replace(',', '')))
                
            except:
                #car['price'] = "0"
                price = "0"
            try:
                colour = car_html.select('section.result-features.features-grid')[0].select('ul')[1].select('li')[0].text.replace('Paint','')                                                
            except:
                colour ="N/A"
            try:
                car_type = car_html.select_one('section.result-header').select_one('div.result-basic-info').select_one('h3').text
            except:
                car_type = "N/A"
            try:    
                trim = car_html.select_one('section.result-header').select_one('div.result-basic-info').select('div')[0].text
            except:
                trim = "N/A"
            try:
                mileage = int(car_html.select_one('section.result-header').select_one('div.result-basic-info').select('div')[1].text.replace('Less than ', '').replace(' mile odometer', '').replace(',', ''))
            except:
                mileage = "0"
            try:
                location = car_html.select_one('section.result-header').select_one('div.result-basic-info').select('div')[2].text
            except:
                location = "N/A"
            try:
                wheels = re.sub('[^0-9]', '', car_html.select('section.result-features.features-grid')[0].select('ul')[1].select('li')[1].text) + " inch wheels"
            except:
                wheels = "N/A"
            try:
                interior = car_html.select('section.result-features.features-grid')[0].select('ul')[1].select('li')[2].text
            except:
                interior = "N/A"
          
                
        #Append collected data into a list
            car['price'].append(price)
            car['colour'].append(colour)
            car['car_type'].append(car_type[5:])
            car['year'].append(car_type[0:4])
            car['trim'].append(trim)
            car['mileage'].append(mileage)
            car['location'].append(location)
            car['wheels'].append(wheels)
            car['interior'].append(interior)
           
        
        time.sleep(30) # seconds
print("Data Scrapped")




04:18:36 Searching Tesla's website in https://www.tesla.com/inventory/used/ms?TRIM=MSPLAID,MSAWD,MSPERF,75D&arrangeby=plh&zip=94555


C:\Users\STSC\AppData\Local\Temp\ipykernel_16664\973004778.py:46: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:/Users/STSC/OneDrive - horizon.csueastbay.edu/Documents/Semester II/Data Analytics/Project 2023/chromedriver.exe', options=chrome_options)


04:19:18 Searching Tesla's website in https://www.tesla.com/inventory/used/mx?TRIM=75D&arrangeby=plh&zip=94555
04:19:58 Searching Tesla's website in https://www.tesla.com/inventory/used/my?arrangeby=plh&zip=94555
04:20:38 Searching Tesla's website in https://www.tesla.com/inventory/used/m3?arrangeby=plh&zip=94555
Data Scrapped


In [6]:
import pandas as pd
df=pd.DataFrame.from_dict(car, orient='index') #convert dictionary to dataframe and add Nan to unfilled values
df=df.T #transpose dataframe
#Data Cleaning
#drop all null and rest index
df = df.dropna()
df.reset_index(inplace=True, drop=True)
df.to_csv('Tesla_Inventory.csv') #Convert dataframe to csv file